# Predict Position of Player with Seasonal Stats
Author: Sarah Thayer

Course Project, UC Irvine, Math 10, W22

## Questions to help Chris

(Chris will remove this section if you want your project posted on the course notes.)

The course [project's instructions](https://christopherdavisuci.github.io/UCI-Math-10-W22/Proj/CourseProject.html) are posted in the course notes and so are the instructions for [Homework 8](https://christopherdavisuci.github.io/UCI-Math-10-W22/Week9/Homework8.html).

* What is your Student ID number?

* What is the "extra component" (or multiple extra components) that you're including in this project that weren't covered in Math 10?

* Would you like your project posted in the course notes?  (This might make the project easier to share if you are applying for an internship, for example.  Feel free to include additional pieces above if you want, such as your email address or a link to your homepage or GitHub repository, if you have one.)

* If a lot of your knowledge of Python comes from somewhere outside of Math 10 (e.g., you took another Python class before), tell me a little about that.  (This way, if your project looks significantly different from what we learned in Math 10, this explanation will help me understand why.)

* Student ID: 42447968

# Extra component: Clustering with KNN or karas. 

* Yes, I would like my finished project in the course notes, thank you. 

* I have not studied python before math 10, I will add any references I use.

## Introduction

My project is exploring two Kaggle Datasets of NBA stats to predict their listed position. 
Two datasets are needed. One contains the players and their listed positions. The second contains season stats, height, and weight. We. merge the two for a complete dataset.

## Main portion of the project

(You can either have all one section or divide into multiple sections)

In [ ]:
import numpy as np
import pandas as pd
import altair as alt

#### Prepare First Dataset 

NBA Stats [found here](https://www.kaggle.com/mcamli/nba17-18?select=nba.csv). 
The columns we are interested in extracting are `Player`, `Team`, `Pos`, `Age`.

Display shape of the dataframe to confirm we have enough data points.

In [ ]:
df_position = pd.read_csv("nba.csv", sep = ',' ,encoding = 'latin-1')
df_position = df_position.rename(columns={'Tm' : 'Team'}) # Second dataset naming convenetion

df_position = df_position.loc[:,df_position.columns.isin(['Player', 'Team', 'Pos', 'Age']) ]
df_position.head()

,Player,Pos,Age,Team
0,Alex Abrines\abrinal01,SG,24,OKC
1,Quincy Acy\acyqu01,PF,27,BRK
2,Steven Adams\adamsst01,C,24,OKC
3,Bam Adebayo\adebaba01,C,20,MIA
4,Arron Afflalo\afflaar01,SG,32,ORL


In [ ]:
df_position.shape

(664, 4)

#### Clean First Dataset
NBA players that have been traded mid-season appear twice. Drop `Player` duplicates from the dataframe and check shape. 

NBA Players have unique player id in `Player` column. Remove player ID to view names.

In [ ]:
# "Alex Abrines\abrinal01" remove the unique player id after the "\"
df_position['Player'] = df_position['Player'].map(lambda x: x.split('\\')[0])



df_pos_unique = df_position[~df_position.duplicated(subset=['Player'])]
df_pos_unique.shape

~df_position.duplicated(subset=['Player'])

0       True
1       True
2       True
3       True
4       True
       ...  
659    False
660    False
661     True
662     True
663     True
Length: 664, dtype: bool

In [ ]:
df_position.iloc[659]

Player    Tyler Zeller
Pos                  C
Age                 28
Team               BRK
Name: 659, dtype: object

In [ ]:
df_pos_unique[df_pos_unique['Player'] == 'Tyler Zeller']

,Player,Pos,Age,Team
658,Tyler Zeller,C,28,TOT


In [ ]:
s = df_position.drop_duplicates(subset="Player", keep='first')
s.shape

(540, 4)

#### Prepare Second Dataset

NBA stats [found here](https://www.kaggle.com/jacobbaruch/basketball-players-stats-per-season-49-leagues).

Large dataset of 20 years of stats form 49 differrent leagues. Parse dataframe for the the relevant data in the NBA leaguerduring the 2017 - 2018 regular season. Then our new dataframe contains height and weight.

In [ ]:
df = pd.read_csv("players_stats_by_season_full_details.csv",  encoding='latin-1' ) 

df= df[(df["League"] == 'NBA') &  (df["Season"] == '2017 - 2018') & (df["Stage"] == 'Regular_Season')] 

df_hw = df.loc[:,~df.columns.isin(['Rk', 'League', 'Season', 'Stage', 'birth_month', 'birth_date', 'height', 'weight',
                                  'nationality', 'high_school', 'draft_round', 'draft_pick', 'draft_team'])]

df_hw.shape

(279, 22)

#### Clean Second Dataset
Drop duplicates from dataframe with 'Player' from the dataframe containing height and weight.

In [ ]:
df_hw_unique = df_hw[~df_hw.duplicated(subset=['Player'])]
df_hw_unique.shape

(279, 22)

#### Prepare Merged Data
- Merge First and Second Dataset 
- Encode the NBA listed positions 


Confirm it's the same player by matching name and team.

In [ ]:
df_merged = pd.merge(df_pos_unique,df_hw_unique, on = ['Player','Team'])

#### One-Hot Encoding
Encode the positions (`strings`) into numbers (`ints`)

In [ ]:
enc = {'PG' : 1, 'SG' : 2,  'SF': 3, 'PF': 4, 'C':5}
df_merged["Pos_enc"] = df_merged["Pos"].map(enc)
df_merged

,Player,Pos,Age,Team,GP,MIN,FGM,FGA,3PM,3PA,...,DRB,REB,AST,STL,BLK,PTS,birth_year,height_cm,weight_kg,Pos_enc
0,Alex Abrines,SG,24,OKC,75,1134.2,115,291,84,221,...,88,114,28,38,8,353,1993.0,198.0,86.0,2
1,Quincy Acy,PF,27,BRK,70,1359.2,130,365,102,292,...,217,257,57,33,29,411,1990.0,201.0,109.0,4
2,Steven Adams,C,24,OKC,76,2487.0,448,712,0,2,...,301,685,88,92,78,1056,1993.0,213.0,120.0,5
3,Bam Adebayo,C,20,MIA,69,1368.1,174,340,0,7,...,263,381,101,32,41,477,1997.0,208.0,116.0,5
4,LaMarcus Aldridge,C,32,SAS,75,2508.7,687,1347,27,92,...,389,635,152,43,90,1735,1985.0,211.0,120.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,Lou Williams,SG,31,LAC,79,2589.2,582,1337,186,518,...,158,198,417,85,19,1782,1986.0,185.0,79.0,2
230,Justise Winslow,PF,21,MIA,68,1680.2,207,488,49,129,...,306,370,148,54,33,529,1996.0,201.0,102.0,4
231,Delon Wright,PG,25,TOR,69,1432.7,201,432,56,153,...,153,198,200,72,33,555,1992.0,196.0,83.0,1
232,Nick Young,SG,32,GSW,80,1393.1,201,488,123,326,...,105,125,36,38,7,581,1985.0,201.0,95.0,2


#### Find Best Model

Feature Selection: Data has player name, team, position, height, weight, and 20+ seasonal stats. Not all  features are relevant to predicting NBA Position. Test with different varations of features. Iterate through `combinations()` of k columns in `cols`. Combinations and estimating the counts of the training trials can be found here: ["...the number of k-element subsets (or k-combinations) of an n-element set"](https://en.wikipedia.org/wiki/Binomial_coefficient).

KNN Optimization: On each combination of possible training features, iterate through range of ints for possible `n_neighbors`.

Log Results: Create results dictionary to store `features`, `n_neighbors`, `log_loss`, and `classifier` for each training iteration. Iterate through results dictionary to find smallest log_loss along with features used, n_neighbors used, and the classifier object.

In [ ]:
# try to hide pandas warnings
# %%capture --no-display

# hide pandas warnings
# https://stackoverflow.com/questions/15777951/how-to-suppress-pandas-future-warning
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import log_loss


from itertools import combinations
cols = ['Pos_enc','FGM', 'FGA', '3PM','3PA','FTM', 'FTA','TOV','PF','ORB','DRB',
        'REB','AST','STL', "BLK",'PTS','height_cm','weight_kg'
]

model_count = 0 # model number we are training

# boolean flags for searching for model
found_clf = False # we found accurate classifier
search_ll_min = True # searching for small log_loss 
search_n = True # searching for ideal n_neighbors
search_clf = True # searching for classifier

# save results in dictionary object
results = {} # key = 
# key = current trial number ("count" or "index" of models)
# values = mapped results. features, n, log_loss, and classifier object.
# {
#     model_count: {
#         'features': (list),
#         'n_neighbors': (int),
#         'log_loss' (float):
#         'classifier': clf object,
#     }
# }


for count in range(14, 16): # 12, 18
    
    # test combinations of columns: test `count` combination of `cols`
    print(f"Testing: {len(cols)} Choose {count}") 
    
    for tup in combinations(cols,count):  # iterate through different combination of columns
       
        for n in range(3,6): # iterate through n neighbors for each combination of columns
            
            if search_n: # if searching for best n_neighbor
                X = df_merged[list(tup)]
                y = df_merged["Pos"]
                scaler = StandardScaler()
                scaler.fit(X)
                X_scaled = scaler.transform(X)
                X_scaled_train, X_scaled_test, y_train, y_test = train_test_split(X,y,test_size=0.2)            

                clf = KNeighborsClassifier(n_neighbors=n)
                clf.fit(X_scaled_train,y_train)
                X["pred"] = clf.predict(X_scaled)

                probs = clf.predict_proba(X_scaled_test)
                loss = log_loss( y_true=y_test ,y_pred=probs , labels= clf.classes_)

                # update results object 
                # key = currential trial number
                # values = mapped results. features, n, log_loss, and classifier object.
                results[model_count] = {
                    'features': list(tup) ,
                    'n_neighbors': n,
                    'log_loss': loss,
                    'classifier': clf
                }

                model_count+=1
                if loss < .7: # if we found accurate classifier 
                    # update boolean flags to stop iterating through columns and n_neighbors
                    search_n = False
                    search_ll_min = False
                    search_clf = False
                    print(f"Found ideal n_neighbors")
                    break
        
        if (search_n == False) or (loss<.6): # found best loss with n_neighbors
            search_ll_min = False
            print('Found combination of features')
            break # keeps the accurate clf

    if not search_ll_min: # found best loss with column combination
        print('return classifier')
        break # return classifier

if not found_clf:
    print(f"couldn't find accurate classifier. Continue to find best results.")

Testing: 18 Choose 14
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

Traceback (most recent call last):
  File "/shared-libs/python3.7/py-core/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3524, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-12783198a0a9>", line 61, in <module>
    X["pred"] = clf.predict(X_scaled)
  File "/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/neighbors/_classification.py", line 228, in predict
    mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
  File "/shared-libs/python3.7/py/lib/python3.7/site-packages/scipy/stats/stats.py", line 448, in mode
    modes[ind], counts[ind] = _mode1D(a_view[ind])
  File "/shared-libs/python3.7/py/lib/python3.7/site-packages/scipy/stats/stats.py", line 434, in _mode1D
    vals, cnts = np.unique(a, return_counts=True)
  File "<__array_function__ internals>", line 6, in unique
  File "/shared-

TypeError: object of type 'NoneType' has no len()

#### Return Best Results

Find the training iteration with the best log loss. 

Return the classifier and print the features selected, neighbors used, and corresponding log loss.

In [ ]:
min_log_loss = 20 # start with bad log_loss
min_log_loss = results[0]['log_loss']
for key in results:
    # key = trial number
    iter_features = results[key]['features']
    iter_n_neighbors = results[key]['n_neighbors']
    iter_log_loss = results[key]['log_loss']
    
    if iter_log_loss < min_log_loss:
        min_log_loss = iter_log_loss
        min_key=key

print(f"attempts: {len(results)}")
print(f"best log_loss: {results[min_key]['log_loss']}")
print(f"best features: {results[min_key]['features']}")
print(f"number of features: {len(results[min_key]['features'])}")
print(f"ideal n_neighbors: {results[min_key]['n_neighbors']}")
print(f"best classifier: {results[min_key]['classifier']}")

attempts: 1962
best log_loss: 0.6524195566464451
best features: ['Pos_enc', 'FGM', 'FGA', '3PM', '3PA', 'FTA', 'ORB', 'DRB', 'REB', 'AST', 'STL', 'BLK', 'height_cm', 'weight_kg']
number of features: 14
ideal n_neighbors: 5
best classifier: KNeighborsClassifier()


Predict position of NBA players on entire dataset.

In [ ]:
X = df_merged[results[min_key]['features']]
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)

clf = results[min_key]['classifier'] # best classifier

df_merged['Preds'] = clf.predict(X) # create predictions for our entire dataset (to see resutls of entire NBA)

#### Vizualize Results

Display all the Centers. Our predicted values show we're pretty good at identifying Centers.

Look at a true Point Gaurd. Chris Paul is a good example of a Point Gaurd.

Look at Lebron James. In 2018 for Clevland Kaggle has him listed has a PF. 

In [ ]:
df_merged[df_merged['Pos_enc']==5].head() # all centers

,Player,Pos,Age,Team,GP,MIN,FGM,FGA,3PM,3PA,...,REB,AST,STL,BLK,PTS,birth_year,height_cm,weight_kg,Pos_enc,Preds
2,Steven Adams,C,24,OKC,76,2487.0,448,712,0,2,...,685,88,92,78,1056,1993.0,213.0,120.0,5,C
3,Bam Adebayo,C,20,MIA,69,1368.1,174,340,0,7,...,381,101,32,41,477,1997.0,208.0,116.0,5,C
4,LaMarcus Aldridge,C,32,SAS,75,2508.7,687,1347,27,92,...,635,152,43,90,1735,1985.0,211.0,120.0,5,C
5,Jarrett Allen,C,19,BRK,72,1441.2,234,397,5,15,...,388,49,28,88,587,1998.0,211.0,108.0,5,C
16,Aron Baynes,C,31,BOS,81,1484.7,210,446,3,21,...,434,93,22,51,482,1986.0,208.0,118.0,5,C


In [ ]:
df_merged[df_merged['Player']=='Chris Paul'] # true point guard

,Player,Pos,Age,Team,GP,MIN,FGM,FGA,3PM,3PA,...,REB,AST,STL,BLK,PTS,birth_year,height_cm,weight_kg,Pos_enc,Preds
178,Chris Paul,PG,32,HOU,58,1846.6,367,798,144,379,...,313,457,96,14,1081,1985.0,183.0,79.0,1,PG


In [ ]:
df_merged[df_merged['Player']=='LeBron James'] # 2018 position correcrt. He's now PG/SF for Lakers

,Player,Pos,Age,Team,GP,MIN,FGM,FGA,3PM,3PA,...,REB,AST,STL,BLK,PTS,birth_year,height_cm,weight_kg,Pos_enc,Preds
110,LeBron James,PF,33,CLE,82,3025.8,857,1580,149,406,...,709,747,116,71,2251,1984.0,203.0,113.0,4,PF


#### Evaluate Performance
Evalute the performance of your classifier using the `log_loss` metric 

In [ ]:
X_scaled_train, X_scaled_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

probs = clf.predict_proba(X_scaled_test)

log_loss(y_true=y_test ,y_pred=probs , labels= clf.classes_)
loss = log_loss(y_true=y_test ,y_pred=probs , labels= clf.classes_)
loss # still accurate

0.7275317784323452

1st row mostly Pink some Light Blue. 
We're good at determining Point Guards, sometimes Small Forwards trigger false positive.

5th row mostly Blue with some Yellow.
Makes sense. Great at determining Centers. Sometimes Power Forwards trigger false positive. 

In [ ]:
chart = alt.Chart(df_merged).mark_circle().encode(
    x=alt.X('height_cm:O', axis=alt.Axis(title='Height in cm')),
    y=alt.X('Pos_enc:O', axis=alt.Axis(title='Encoded')),
    color = alt.Color("Preds", title = "Predicted NBA Positions"),
).properties(
    title = f"n_neighbors = {4}",
)
chart

alt.Chart(...)

<b>Chart Annotations</b><br>
Plot players according to position. 
The points along Encoded 5 are all centers. <br>
The colors are what we predicted. <br>
So the all blue row means we did good at predicting centers. <br>

<br>
Row 2 being a blend of different colors shows it's hard to predict a Small Forward by stats. 

![chart_annot.png](attachment:/work/chart_annot.png) 


End

#### Interpreting the results

Plotting by encoded positions we can see which positions are easier to classify. Centers for instance seem to have high true positive results. 

## Summary
Taking player seasonal stats, height, and weight we attempted to predict NBA positions by classification. Some NBA positions are easier to predict than others. 

## References

Include references that you found helpful.  Also say where you found the dataset you used.

##### Dataframes used from Kaggle


Basketball Players Stats per Season - 49 Leagues [found here](https://www.kaggle.com/jacobbaruch/basketball-players-stats-per-season-49-leagues).

NBA Player Stats 2017-2018 [found here](https://www.kaggle.com/mcamli/nba17-18?select=nba.csv). 


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5ea705d8-ef20-4907-b1ee-133b5616033e' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>